<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Diego Pintor Ochoa, Lyha Espinosa García

**Fecha:** 28 de abril de 2021.

**Expediente** : if72622, .
**Profesor:** Oscar David Jaramillo Zuluaga.

# Tarea 9: Clase 21

## Enunciado de tarea 
> Replicar el procedimiento anterior para valoración de opciones 'call', pero en este caso para opciones tipo 'put' y compararlo con el valor teórico de la ecuación de Black-Scholes.

### Solución Lyha Espinosa

In [ ]:
# Código de solución 
.
.
.
.


### Solución Diego Pintor


In [ ]:
# Código de solución 
.
.
.
.
